## Initialisation des fonctions pour le netoyage

In [4]:
# Besoin pour clean_mileage_engine_power
import pandas as pd
import numpy as np


name_info2del = {
    # 'Maruti' : set(['(abs)', '(cng)', 'Airbag', 'Aktiv', 'Alpha', 'DX', 'DX2', 'Immobiliser', 'With']),
    # 'Hyundai': set(['(0)', '(o)', 'With']),
    # 'Honda'  : set(['(at)', '(e)']),
    # 'Audi'   : set(['(at)', '(e)']),
}




def to_pascal_case(str):
    words = str.split()
    pascal_case_words = [word.capitalize() for word in words]
    pascal_case_string = ''.join(pascal_case_words)
    return pascal_case_string



def clean_mileage(mileage):
    if pd.isna(mileage):  # Gérer les valeurs NaN
        return np.nan
    num, unit = mileage.split(' ')[0], mileage.split(' ')[1]
    num = float(num)
    if unit == 'km/kg':
        return num * 1.40
    elif unit == 'kmpl':
        return num
    else:
        return np.nan
    
def clean_engine(engine):
    if pd.isna(engine):
        return np.nan
    num, unit = engine.split(' ')[0], engine.split(' ')[1]
    num = float(num)
    if unit == 'CC':
        return num
    
def clean_power(power):
    if pd.isna(power) or power.split(' ')[0].lower() == 'null':
        return np.nan
    num, unit = power.split(' ')[0], power.split(' ')[1]
    try:
        num = float(num)
        if unit == 'bhp':
            return num
    except ValueError:
        return np.nan

def clean_name(df):
    df.insert(1, 'Version', '')

    for index in df.index:
        name = df.at[index, 'Name']
        ws_idx = name.find(' ')
        vehicle_model = name[:ws_idx]
        vehicle_version = name[ws_idx+1:]
        vv_split = list(map(lambda s: to_pascal_case(s), vehicle_version.split(' ')))

        if (vehicle_model in name_info2del):
            formated_version = (set(vv_split) - name_info2del[vehicle_model])
        else:
            formated_version = set(vv_split)
        #formated_version -= name_info2del_default
        formated_version = ' '.join(sorted(list(formated_version)))

        df.at[index, 'Name'] = vehicle_model
        df.at[index, 'Version'] = formated_version

    return df

## Composée des fonctions

In [19]:
def clean_csv(df):
    df['Mileage'] = df['Mileage'].apply(clean_mileage)
    df['Engine'] = df['Engine'].apply(clean_engine)
    df['Power'] = df['Power'].apply(clean_power)
    df = clean_name(df)
    return df

## Génération du fichier CSV

In [20]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

clean_train = clean_csv(data_train)
clean_train.to_csv('mimified_train.csv', index=False)

clean_test = clean_csv(data_test)
clean_test.to_csv('mimified_test.csv', index=False)